In [1]:
import numpy as np
import gc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

# Data pre-prep; Don't run again

In [3]:
df1 = pd.read_csv('data/csgo_price_hist.csv')
df2 = pd.read_csv('data/csgo_price_hist2.csv')

In [4]:
duplicate_check = df1.merge(df2, on=list(df1.columns), how="inner")
if duplicate_check.shape[0] == 0:
  print('No duplicates across df1 and df2')

No duplicates across df1 and df2


In [6]:
def agg_hist_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Group the dataframe by 'market', 'market_hash_name', and 'dates.date'.
    Sums the 'quantity' and takes the minimum of 'price'.

    Parameters:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The aggregated DataFrame.
    """
    return df.groupby(['market_hash_name', 'dates.date']).agg({
        'quantity': 'sum',
        'price': 'min'
    }).reset_index()


agg_df1 = agg_hist_data(df1)
agg_df2 = agg_hist_data(df2)

print(agg_df1.shape)
print(agg_df2.shape)

(192366, 4)
(7132894, 4)


In [9]:
del df1
del df2
gc.collect()

In [10]:
agg_df_hist = pd.concat([agg_df1, agg_df2])
agg_df_hist.to_csv('data/csgo_agg_df_hist.csv')
agg_df_hist.shape

(7325260, 4)

In [12]:
agg_df_hist.sort_values('dates.date', ascending=False).head()

,market_hash_name,dates.date,quantity,price
7132893,★ Ursus Knife | Urban Masked (Well-Worn),2023-09-19,50,12593
1343707,Pinups Capsule,2023-09-19,2359,81
2747607,StatTrak™ AK-47 | Rat Rod (Battle-Scarred),2023-09-19,247,762
5936742,USP-S | Guardian (Minimal Wear),2023-09-19,1833,339
1344071,Poorly Drawn Capsule,2023-09-19,722,80


In [13]:
agg_df_hist = pd.read_csv('data/csgo_agg_df_hist.csv', index_col=0)
agg_df_hist.price = agg_df_hist.price/100  # Get data in USD (isntead of USD cents)
agg_df_hist = agg_df_hist.loc[(agg_df_hist['dates.date'] > '2022-09-20') & 
                              (agg_df_hist['dates.date'] < '2023-09-19'), :]

In [14]:
fluctuation = agg_df_hist.groupby('market_hash_name').agg({
    'quantity': ['std', 'mean', 'min', 'max',
                 lambda x: x.quantile(0.10),
                 lambda x: x.quantile(0.25),
                 lambda x: x.quantile(0.50),
                 lambda x: x.quantile(0.75),
                 lambda x: x.quantile(0.90)],
    'price': ['mean']
}).reset_index()

fluctuation.columns = [
    'market_hash_name',
    'std_quantity',
    'mean_quantity',
    'min_quantity',
    'max_quantity',
    '10th_percentile_quantity',
    '25th_percentile_quantity',
    'median',
    '75th_percentile_quantity',
    '90th_percentile_quantity',
    'mean_price'
]

fluctuation['coef_of_variation'] = fluctuation['std_quantity'] / fluctuation['mean_quantity']
fluctuation.head()


,market_hash_name,std_quantity,mean_quantity,min_quantity,max_quantity,10th_percentile_quantity,25th_percentile_quantity,median,75th_percentile_quantity,90th_percentile_quantity,mean_price,coef_of_variation
0,'Medium Rare' Crasswater | Guerrilla Warfare,306.448272,1000.168044,236,1589,674.0,771.0,908.0,1313.5,1428.8,8.491983,0.306397
1,'The Doctor' Romanov | Sabre,301.795136,3281.994490,2025,3901,2889.8,3077.0,3268.0,3503.5,3711.0,3.829394,0.091955
2,'Two Times' McCoy | TACP Cavalry,448.358181,1454.044077,711,2445,904.0,1095.5,1340.0,1831.5,2111.8,1.686722,0.308353
3,'Two Times' McCoy | USAF TACP,574.310914,1993.162534,698,3231,1267.4,1546.5,1866.0,2465.0,2799.0,1.639339,0.288141
4,10 Year Birthday Sticker Capsule,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265


In [15]:
agg_df_hist = agg_df_hist.merge(fluctuation, on='market_hash_name', how='inner')
agg_df_hist.to_csv('data/csgo_agg_df_hist_fluct.csv')

# Start here

In [3]:
agg_df_hist = pd.read_csv('data/csgo_agg_df_hist_fluct.csv', index_col=0)
print('shape: ', agg_df_hist.shape)
agg_df_hist.head()

shape:  (7304522, 15)


,market_hash_name,dates.date,quantity,price,std_quantity,mean_quantity,min_quantity,max_quantity,10th_percentile_quantity,25th_percentile_quantity,median,75th_percentile_quantity,90th_percentile_quantity,mean_price,coef_of_variation
0,10 Year Birthday Sticker Capsule,2022-09-21,3620,0.74,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
1,10 Year Birthday Sticker Capsule,2022-09-22,3465,0.73,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
2,10 Year Birthday Sticker Capsule,2022-09-23,3364,0.73,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
3,10 Year Birthday Sticker Capsule,2022-09-24,2906,0.73,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
4,10 Year Birthday Sticker Capsule,2022-09-25,2460,0.74,2651.694205,5343.300275,1260,12531,2089.4,3294.0,4782.0,7224.5,9408.6,0.768457,0.496265
